<a href="https://colab.research.google.com/github/Alecia113/NLP-Ex/blob/main/lab5_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [2]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1gNfBqguzBu8cHKMPc8C44GbvD443dNC5'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('twitter.csv')  

import pandas as pd
df = pd.read_csv("twitter.csv")
df_pick = df.sample(400,random_state=24)

raw_text = df_pick["Text"].tolist()
raw_label = df_pick["Label"].tolist()

from sklearn.model_selection import train_test_split
text_train,text_test,label_train,label_test = train_test_split(raw_text,raw_label,test_size=0.25,random_state=42)

In [3]:
text_train = [s.lower() for s in text_train]
text_test = [s.lower() for s in text_test]

In [4]:
import re
def remove_punctuation_re(x):
    # Please complete this
    x = re.sub(r'[^\w\s]','',x)
    return x
    
text_train = [remove_punctuation_re(s) for s in text_train]
text_test = [remove_punctuation_re(s) for s in text_test]

In [ ]:
import nltk
nltk.download('punkt')
import pprint
from nltk.tokenize import word_tokenize

train_t=[]
for s in text_train:
  text_train = word_tokenize(s)                         #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
pprint.pprint(train_t[:10])             #目前是300 #[['finalbroadcast', 'or', 'many', 'dkim', 'setups', 'for', 'that', 'matter'],

test_t=[]
for w in text_test:
  text_test = word_tokenize(w)
  test_t.append(text_test)
#pprint.pprint(text_test[1:3])

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()

text_train_ns=[]
for tokens in train_t:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_ns.append(filtered_sentence1)

text_test_ns=[]
for tokens in test_t:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_ns.append(filtered_sentence2)
    #Please complete this

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text_train_le = []
for tokens in text_train_ns:
  lemma_sentence1 = [lemmatizer.lemmatize(w) for w in tokens ]
  text_train_le.append(lemma_sentence1)

text_test_le = []
for tokens in text_test_ns:
  lemma_sentence2 = [lemmatizer.lemmatize(w) for w in tokens]
  text_test_le.append(lemma_sentence2)
    #Please complete this


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pprint.pprint(text_train_le[:10])
pprint.pprint(text_test_le[:10])

In [9]:
import numpy as np

from sklearn.preprocessing import LabelEncoder

unique_labels = np.unique(label_train)

lEnc = LabelEncoder()
# Please encode the labels (Do NOT add new lines of code in this section)
# Hint: Try to understand the difference between fit_transform and transform
label_train_encoded = lEnc.fit_transform(label_train)
label_test_encoded = lEnc.fit_transform(label_test)

n_class = len(unique_labels)
print(unique_labels)
print(lEnc.transform(unique_labels))

['none' 'racism' 'sexism']
[0 1 2]


In [10]:
word_set = set() 
for sent in text_train_le:
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index)

['0', '06jank', '0xjared', '1', '11', '12', '14', '1400', '15', '17', '1shadeofritch', '2', '2027279099', '22000', '2ndbestidiot', '3', '3outof10', '4', '44', '47', '4x', '5', '6', '7', '80', '800', '90', '911', '98halima', '99', '[PAD]', '[UNKNOWN]', '__chris33__', '_marisajane', 'abducted', 'abdul_a95', 'aberration', 'ability', 'ablahad', 'able', 'absolutely', 'abuse', 'ac360', 'accept', 'acceptable', 'accepted', 'accessorizing', 'account', 'achieve', 'across', 'actoractress', 'actually', 'adjective', 'admits', 'adult', 'afar', 'afraid', 'ago', 'agree', 'ahahahaha', 'air', 'airstrikes', 'aisle', 'ajwatamr', 'akheemv', 'aledthomas22', 'alihadi68', 'alihashem_tv', 'alive', 'all_hailcaesar', 'allegedly', 'allstatejackie', 'ally', 'along', 'already', 'alternet', 'amaze', 'amazing', 'amazingly', 'amberhasalamb', 'ameliagreenhall', 'american', 'amohedin', 'amp', 'amymek', 'anasmechch', 'andcamping', 'andre', 'angelemichelle', 'anitaingle', 'annoying', 'another', 'answer', 'anti', 'antihara

In [16]:

# The sequence length is pre-defined, you can't change this value for this exercise
seq_length = 16

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_mbadd_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 

In [18]:
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-50") # Download an embedding other than glove-twitter-25

[==================================================] 100.0% 199.5/199.5MB downloaded


In [21]:
# Get the Embedding lookup table
import numpy as np
emb_dim = word_emb_model.vector_size

emb_table = []
for i, word in enumerate(word_list):
    if word in word_emb_model:
        emb_table.append(word_emb_model[word])
    else:
        emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)

In [23]:
vocab_size = len(word_list)
n_hidden = 50
total_epoch = 100
learning_rate = 0.01

In [25]:
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.emb.weight.data.copy_(torch.from_numpy(emb_table))
        self.emb.weight.requires_grad = False
        self.lstm = nn.LSTM(emb_dim, n_hidden, num_layers=2, batch_first =True, dropout=0.2)
        self.linear = nn.Linear(n_hidden,n_class)

    def forward(self, x):
        x = self.emb(x)
        x,_ = self.lstm(x)
        x = self.linear(x[:,-1,:])
        return x


model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

input_torch = torch.from_numpy(np.array(train_pad_encoded)).to(device)
target_torch = torch.from_numpy(np.array(label_train_encoded)).view(-1).to(device)



for epoch in range(total_epoch):  
    
    model.train()
    optimizer.zero_grad()
    outputs = model(input_torch) 
    loss = criterion(outputs, target_torch) #[1,3][300]
    loss.backward()
    optimizer.step()
    
    if epoch%10 == 9:
        predicted = torch.argmax(outputs, -1)
        acc= accuracy_score(predicted.cpu().numpy(),target_torch.cpu().numpy())

        print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

print('Finished Training')


Epoch: 10, loss: 0.73664, train_acc: 0.67
Epoch: 20, loss: 0.62510, train_acc: 0.70
Epoch: 30, loss: 0.44414, train_acc: 0.80
Epoch: 40, loss: 0.30211, train_acc: 0.89
Epoch: 50, loss: 0.25571, train_acc: 0.93
Epoch: 60, loss: 0.20955, train_acc: 0.94
Epoch: 70, loss: 0.16944, train_acc: 0.95
Epoch: 80, loss: 0.22171, train_acc: 0.94
Epoch: 90, loss: 0.15814, train_acc: 0.96
Epoch: 100, loss: 0.14234, train_acc: 0.96
Finished Training


In [26]:
torch.save(model,'lab5.pt')

In [27]:
model2 = torch.load('lab5.pt')
model2.eval()

Model(
  (emb): Embedding(1640, 50)
  (lstm): LSTM(50, 50, num_layers=2, batch_first=True, dropout=0.2)
  (linear): Linear(in_features=50, out_features=3, bias=True)
)

In [28]:
input_torch = torch.from_numpy(np.array(test_pad_encoded)).to(device)

outputs = model2(input_torch) 
predicted = torch.argmax(outputs, -1)

from sklearn.metrics import classification_report
print(classification_report(label_test_encoded,predicted.cpu().numpy()))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75        65
           1       0.70      0.70      0.70        10
           2       0.48      0.48      0.48        25

    accuracy                           0.68       100
   macro avg       0.64      0.64      0.64       100
weighted avg       0.68      0.68      0.68       100

